# WeatherPy

---

## Starter Code to Generate Random Geographic Coordinates and a List of Cities

In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from scipy.stats import linregress
from citipy import citipy
from datetime import datetime
import time

# Impor the OpenWeatherMap API key
from api_keys import weather_api_key

# Define URL
query_url = "https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API key}"

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy



### Generate the Cities List by Using the `citipy` Library

In [19]:
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Empty list for holding the cities names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)


# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"Number of cities in the list: {len(cities)}")

Number of cities in the list: 628


---

## Requirement 1: Create Plots to Showcase the Relationship Between Weather Variables and Latitude

### Use the OpenWeatherMap API to retrieve weather data from the cities list generated in the started code

In [20]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=35e2c5405da038817ec99fa61c83ea38


In [21]:

# Define an empty list to fetch the weather data for each city
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # Create endpoint URL with each city
    city_url = url + "&q=" + city 
    
    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()

        # Parse out latitude, longitude, max temp, humidity, cloudiness, wind speed, country, and date
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_humidity = city_weather['main']['humidity']
        city_clouds = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_country = city_weather['sys']['country']
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')

        # Append the City information into city_data list
        city_data.append({"city": city, 
                          "lat": city_lat, 
                          "lng": city_lng, 
                          "max_Temp": city_max_temp,
                          "humidity": city_humidity,
                          "cloudiness": city_clouds,
                          "wind_Speed": city_wind,
                          "country": city_country,
                          "date": city_date})

    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
              
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | atuona
Processing Record 2 of Set 1 | cape town
Processing Record 3 of Set 1 | jamestown
Processing Record 4 of Set 1 | port hedland
Processing Record 5 of Set 1 | hermanus
Processing Record 6 of Set 1 | pestovo
Processing Record 7 of Set 1 | mount gambier
Processing Record 8 of Set 1 | lorengau
Processing Record 9 of Set 1 | hokitika
Processing Record 10 of Set 1 | vaini
Processing Record 11 of Set 1 | mataura
Processing Record 12 of Set 1 | busselton
Processing Record 13 of Set 1 | provideniya
Processing Record 14 of Set 1 | hobart
Processing Record 15 of Set 1 | khatanga
Processing Record 16 of Set 1 | nizhneyansk
City not found. Skipping...
Processing Record 17 of Set 1 | port elizabeth
Processing Record 18 of Set 1 | bethel
Processing Record 19 of Set 1 | butaritari
Processing Record 20 of Set 1 | steinau
Processing Record 21 of Set 1 | baisha
Processing Record 22 of Set 1 | albany
Processin

In [22]:
# Convert the cities weather data into a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)

# Show Record Count
city_data_df.count()

city          573
lat           573
lng           573
max_Temp      573
humidity      573
cloudiness    573
wind_Speed    573
country       573
date          573
dtype: int64

In [32]:
# Display sample data
city_data_df.head(10)

,city,lat,lng,max_Temp,humidity,cloudiness,wind_Speed,country,date
City_ID,,,,,,,,,
0,atuona,-9.8000,-139.0333,79.43,83,99,13.73,PF,2023-04-03 16:40:44
1,cape town,-33.9258,18.4232,69.35,71,40,4.61,ZA,2023-04-03 16:37:09
2,jamestown,42.0970,-79.2353,60.39,44,0,13.80,US,2023-04-03 16:36:09
3,port hedland,-20.3167,118.5667,80.65,83,37,8.05,AU,2023-04-03 16:44:14
4,hermanus,-34.4187,19.2345,65.80,80,59,8.28,ZA,2023-04-03 16:40:48
5,pestovo,58.5938,35.8024,32.65,98,100,6.08,RU,2023-04-03 16:44:14
6,mount gambier,-37.8333,140.7667,50.58,95,52,4.43,AU,2023-04-03 16:42:35
7,lorengau,-2.0226,147.2712,79.72,77,84,2.80,PG,2023-04-03 16:42:53
8,hokitika,-42.7167,170.9667,50.09,85,7,4.90,NZ,2023-04-03 16:44:15


In [24]:
# Export the City_Data into a csv
city_data_df.to_csv("output_data/cities.csv", index_label="City_ID")

In [25]:
# Read saved data
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

,city,lat,lng,max_Temp,humidity,cloudiness,wind_Speed,country,date
City_ID,,,,,,,,,
0,atuona,-9.8000,-139.0333,79.43,83,99,13.73,PF,2023-04-03 16:40:44
1,cape town,-33.9258,18.4232,69.35,71,40,4.61,ZA,2023-04-03 16:37:09
2,jamestown,42.0970,-79.2353,60.39,44,0,13.80,US,2023-04-03 16:36:09
3,port hedland,-20.3167,118.5667,80.65,83,37,8.05,AU,2023-04-03 16:44:14
4,hermanus,-34.4187,19.2345,65.80,80,59,8.28,ZA,2023-04-03 16:40:48


### Create the Scatter Plots Requested

#### Latitude Vs. Temperature

In [26]:
# Build scatter plot for latitude vs. temperature
# YOUR CODE HERE

# Incorporate the other graph properties
# YOUR CODE HERE

# Save the figure
plt.savefig("output_data/Fig1.png")

# Show plot
plt.show()

<Figure size 640x480 with 0 Axes>

#### Latitude Vs. Humidity

In [27]:
# Build the scatter plots for latitude vs. humidity
# YOUR CODE HERE

# Incorporate the other graph properties
# YOUR CODE HERE

# Save the figure
plt.savefig("output_data/Fig2.png")

# Show plot
plt.show()

<Figure size 640x480 with 0 Axes>

#### Latitude Vs. Cloudiness

In [28]:
# Build the scatter plots for latitude vs. cloudiness
# YOUR CODE HERE

# Incorporate the other graph properties
# YOUR CODE HERE

# Save the figure
plt.savefig("output_data/Fig3.png")

# Show plot
plt.show()

<Figure size 640x480 with 0 Axes>

#### Latitude vs. Wind Speed Plot

In [29]:
# Build the scatter plots for latitude vs. wind speed
# YOUR CODE HERE

# Incorporate the other graph properties
# YOUR CODE HERE

# Save the figure
plt.savefig("output_data/Fig4.png")

# Show plot
plt.show()

<Figure size 640x480 with 0 Axes>

---

## Requirement 2: Compute Linear Regression for Each Relationship


In [30]:
# Define a function to create Linear Regression plots
# YOUR CODE HERE

In [31]:
# Create a DataFrame with the Northern Hemisphere data (Latitude >= 0)
# YOUR CODE HERE

# Display sample data
northern_hemi_df.head()

NameError: name 'northern_hemi_df' is not defined

In [ ]:
# Create a DataFrame with the Southern Hemisphere data (Latitude < 0)
# YOUR CODE HERE

# Display sample data
southern_hemi_df.head()

###  Temperature vs. Latitude Linear Regression Plot

In [ ]:
# Linear regression on Northern Hemisphere
# YOUR CODE HERE

In [ ]:
# Linear regression on Southern Hemisphere
# YOUR CODE HERE

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Humidity vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
# YOUR CODE HERE

In [ ]:
# Southern Hemisphere
# YOUR CODE HERE

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Cloudiness vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
# YOUR CODE HERE

In [ ]:
# Southern Hemisphere
# YOUR CODE HERE

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Wind Speed vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
# YOUR CODE HERE

In [ ]:
# Southern Hemisphere
# YOUR CODE HERE

**Discussion about the linear relationship:** YOUR RESPONSE HERE